In [3]:
import pandas as pd
import numpy as np
import env
import acquire
import os

In [4]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    '''This function uses credentials from an env file to log into a database'''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [5]:
def new_zillow_db():
    '''The function uses the get_connection function to connect to a database and retrieve the zillow dataset'''
    return pd.read_sql('''SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, 
    yearbuilt, taxamount, fips from properties_2017 LEFT JOIN propertylandusetype USING(propertylandusetypeid)
    WHERE propertylandusedesc IN ("Single Family Residential", "Inferred Single Family Residential");''', get_connection('zillow'))

In [6]:
df = new_zillow_db()

In [7]:
def get_zillow_data():
    '''
    This function reads in telco data from Codeup database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('zillow.csv'):
        
        # If csv file exists read in data from csv file.
        df = pd.read_csv('zillow.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame
        df = new_zillow_db()
        
        # Cache data
        df.to_csv('zillow.csv')
        
    return df

In [8]:
df = get_zillow_data()

In [9]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [23]:
#df = df.dropna()

In [24]:
#df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2901918 entries, 2 to 2982283
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 177.1 MB


In [28]:
#df.bedroomcnt = df.bedroomcnt.astype(int)

In [31]:
#df.yearbuilt = df.yearbuilt.astype(int)

In [34]:
#df.fips.value_counts()

6037.0    1970806
6059.0     717971
6111.0     213141
Name: fips, dtype: int64

In [35]:
#df.fips = df.fips.astype(int)

In [37]:
#df = df.rename(columns={'bedroomcnt': 'bedrooms', 'bathroomcnt': 'bathrooms', 'calculatedfinishedsquarefeet': 'sqft', 
#                        'taxvaluedollarcnt': 'tax_value'})

In [39]:
#df = df.rename(columns={'yearbuilt': 'year', 'taxamount': 'tax_amount'})

In [40]:
#df

,bedrooms,bathrooms,sqft,tax_value,year,tax_amount,fips
2,0,0.0,73026.0,1434941.0,1959,20800.37,6037
3,0,0.0,5068.0,1174475.0,1948,14557.57,6037
4,0,0.0,1776.0,440101.0,1947,5725.17,6037
5,0,0.0,2400.0,287634.0,1943,3661.28,6037
7,0,0.0,3611.0,698984.0,1946,7857.84,6037
...,...,...,...,...,...,...,...
2982278,3,2.5,1648.0,538640.0,2014,6370.58,6059
2982280,2,1.0,798.0,469300.0,2006,5764.45,6037
2982281,3,3.0,1526.0,594022.0,2014,7343.47,6037
2982282,4,4.0,2110.0,554009.0,2014,6761.20,6037


In [10]:
def wrangle_zillow():
    '''This function acquires the zillow dataset from the Codeup database using a SQL query and returns a cleaned
    dataframe from a csv file. Observations with null values are dropped and column names are changed for
    readability. Values expected as integers are converted to integer types (year, bedrooms, fips).'''
    # use the get_zillow_data function to acquire the dataset and save it to a csv
    df = get_zillow_data()
    # drop rows with null values
    df = df.dropna()
    # change bedroom count to an integer
    df.bedroomcnt = df.bedroomcnt.astype(int)
    # change year built to an integer
    df.yearbuilt = df.yearbuilt.astype(int)
    # change fips to an integer
    df.fips = df.fips.astype(int)
    # rename columns for readability
    df = df.rename(columns={'bedroomcnt': 'bedrooms', 'bathroomcnt': 'bathrooms', 'calculatedfinishedsquarefeet': 'sqft', 
                        'taxvaluedollarcnt': 'tax_value', 'yearbuilt': 'year', 'taxamount': 'tax_amount'})
    return df

In [11]:
df = wrangle_zillow()

In [12]:
df

,bedrooms,bathrooms,sqft,tax_value,year,tax_amount,fips
4,4,2.0,3633.0,296425.0,2005,6941.39,6037
6,3,4.0,1620.0,847770.0,2011,10244.94,6037
7,3,2.0,2077.0,646760.0,1926,7924.68,6037
11,0,0.0,1200.0,5328.0,1972,91.60,6037
14,0,0.0,171.0,6920.0,1973,255.17,6037
...,...,...,...,...,...,...,...
2152857,4,4.0,4375.0,422400.0,2015,13877.56,6037
2152859,4,3.0,2262.0,960756.0,2015,13494.52,6059
2152860,4,4.5,3127.0,536061.0,2014,6244.16,6059
2152862,3,2.5,1974.0,424353.0,2015,5302.70,6059


In [13]:
df.describe()

,bedrooms,bathrooms,sqft,tax_value,year,tax_amount,fips
count,2.140235e+06,2.140235e+06,2.140235e+06,2.140235e+06,2.140235e+06,2.140235e+06,2.140235e+06
mean,3.301419e+00,2.240352e+00,1.863194e+03,4.606416e+05,1.960968e+03,5.616711e+03,6.048310e+03
std,9.325998e-01,9.905489e-01,1.221754e+03,6.771576e+05,2.215056e+01,7.814563e+03,2.034491e+01
min,0.000000e+00,0.000000e+00,1.000000e+00,2.200000e+01,1.801000e+03,6.340000e+00,6.037000e+03
25%,3.000000e+00,2.000000e+00,1.258000e+03,1.891660e+05,1.949000e+03,2.540850e+03,6.037000e+03
50%,3.000000e+00,2.000000e+00,1.623000e+03,3.282960e+05,1.958000e+03,4.111470e+03,6.037000e+03
75%,4.000000e+00,3.000000e+00,2.208000e+03,5.346060e+05,1.976000e+03,6.411930e+03,6.059000e+03
max,2.500000e+01,3.200000e+01,9.525760e+05,9.018846e+07,2.016000e+03,1.078102e+06,6.111000e+03
